In [132]:
import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse

In [133]:
# Download the specific page to be able to extract information from it 

page = urllib.request.urlopen('https://enb.iisd.org/enb/vol12/').read()
soup = BeautifulSoup(page)

#Global Variable

month = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}

In [201]:
# Function that extracts a date from a given string by returning a tuple of int (day,month,year)

def extract_date(sdate):
    if(extract_number(sdate) == None ):
        return sdate
    
    m = re.findall('\d{4}|\d{2}|January|February|March|April|May|June|July|August|September|October|November|December|\d{1}',sdate)

    if(len(m)==0):
        d=0
    if(len(m)==5):
        d = (int(m[0]),month[m[1]],int(m[len(m)-1]))
    if(len(m)==4):
        d = (int(m[0]),month[m[2]],int(m[len(m)-1]))
    if(len(m)==3):
        if(m[0] in month.keys()):
            d=(m[1],month[m[0]],m[2])
        else :
            d = (m[0],month[m[1]],m[2])
    
    d_str = str(d[2])+"-"+str(d[1])+"-"+str(d[0])
    
    return d_str

In [188]:
# Function that extracts digit from a given string and return an int 

def extract_number(sname):
    
    for i in sname.split():

        if i.isdigit():
            return int(i)
        

In [181]:
# Function that helps to compose and order the list. Returns an ordered, by date, list who contains all the COPs with their attributes

def compute_list(list_string):
    list_cop = []

    for s in list_string:
        l = s.split('|')
        list_cop.append((extract_number(l[0]),extract_date(l[1]),l[2]))
        
    list_cop.sort(key=lambda a: a[0], reverse=False)

    return list_cop


In [182]:
# Function that extract all the issues for each COPs and return a list in which for each COPs we have the issues corresponding with their number  date and html link 

def extract_details_meetings(soup,meeting_type):
    
    detail_meetings = []
    meeting_num = 1
    for row in soup.find_all("tr"):
        for col in row.find_all('td'):
        
            #Detect a new COP 
            if("Issue" in col.string):
                
                a = row.find_previous_sibling('tr')
                b= a.find_next('th')
                detail = []
                # Variable to help to detect the first issue 
                issue_start = 0
                
                if("</h3>"+meeting_type in str(b) and not("BIS" in str(b))):
                    date_td = col.find_next_sibling('td')
                    
                    while( "<a name=" not in str(date_td.find_next('tr'))):              
                
                        #extract issue number
                        issue = int(re.findall('\d+',col.string)[0])
                        
                        # define the issue type
                        if(issue - issue_start>1):
                            issue_type = 'First'
                        else:
                            issue_type = 'Issue'
                        
                        pdf_td = date_td.find_next_sibling('td')
                        
                        # extract html link
                        html_td = pdf_td.find_next_sibling('td')
                        html = 'https://enb.iisd.org'+html_td.find('a',href=True)['href']
                    
                        # extract date
                        s = date_td.string
                        date = extract_date(s)    
                        
                        #Check if at the end of the webpage and return the final 
                        #list otherwise continue to find new issues
                        if(pdf_td.find_next('tr') == None):
                            break
                        else:
                            col = pdf_td.find_next('tr').find_next('td')
                            date_td = col.find_next_sibling('td')
                            issue_start = issue
                       
                        # add the issue into the list
                        detail.append((issue,date,html,issue_type))
                    
                    # Handle case when we are at the end of the COP and we have the summary
                    if( "<a name=" in str(date_td.find_next('tr'))): 
                    #extract issue number
                        issue = int(re.findall('\d+',col.string)[0])
                        
                        pdf_td = date_td.find_next_sibling('td')
                    #extract html link
                        html_td = pdf_td.find_next_sibling('td')
                        html = 'https://enb.iisd.org'+html_td.find('a',href=True)['href']
                    
                    #extract date
                        s = date_td.string
                        date = extract_date(s)
                        detail.append((issue,date,html,'Summary'))
                    
                    if(pdf_td.find_next('tr') == None):
                            detail.append((issue,date,html,'Summary'))
                            detail_meetings.append(detail)
                            return detail_meetings
                    
                     
                    detail_meetings.append(detail)
                    meeting_num = meeting_num +1
    return detail_meetings
                    

  

In [183]:
# ------------------------------ COP ------------------------------ 

In [155]:
# Function that extract the list of all the COPs from a webpage and return a list containing all the COPs with their number, date and place

def extract_list_cops(soup):
    
    # find all the different COPs (not named the same way)
    
    #Case 1 :"COP"+"\s"+"\d"+"\s"+"."+"\s"  // COP 1-9
    list = soup.find_all(string=re.compile("COP"+"\s"+"\d"+"\s"+"."+"\s"))

    #Case 2 : "COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"\d" // COP 10,23,24,25
    list += soup.find_all(string=re.compile("COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"\d"))
    
    #Case 3 :"COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"CMP"+"\s"+"\d"+"."+"." // COP 11,12,13,14,15,16,21,22
    list_2 = soup.find_all(string=re.compile("COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"CMP"+"\s"+"\d+"+"\s"))

    # Case 4 : "COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"CMP"+"\d"+"\s" //COP 17,18,19
    list_3 = soup.find_all(string=re.compile("COP"+"\s"+"[1-2][0-9]"+"\s"+"."+"\s"+"CMP"+"\d+"+"\s"))
    
    # Clean the lists to have all the same structure
    # Clean list_2
    for i in range(len(list_2)) :
        list_2[i] =  re.sub("- CMP"+"\s"+"."+".", '', list_2[i])

    # Clean list_3
    for i in range(len(list_3)) :
        list_3[i] =  re.sub("- CMP"+".", '', list_3[i])
    
    #combine all the lists
    list += list_2
    list += list_3
   
    return compute_list(list)


In [140]:
# ------------------------------ INC ------------------------------ 

In [141]:
# Function that extract the list of all the INCs from a webpage and return a list containing all the COPs with their number, date and place

def extract_list_incs(soup):
    
    # find all the different INCs (not named the same way)
    # Only one
    
    #Case 1 :"INC"+"\s"+"\d+"+"\s"  // INC 11
    list = soup.find_all(string=re.compile("INC"+"\s"+"\d+"+"\s"))

    return compute_list(list)



In [142]:
# ------------------------------ SB ------------------------------ 

In [143]:
# Function that extract the list of all the SBs from a webpage and return a list containing all the COPs with their number, date and place

def extract_list_sbs(soup):
    
    # find all the different INCs (not named the same way)
    # Only one 
    

    
    #Case 1 :"SB"+"\s"+"\d+"+"\s"+"."+"\s"+"\d+"
    list_1 = soup.find_all(string=re.compile("SB"+"\s"+"\d+"+"\s"+"."+"\s"+"\d+"))
    
    #Case 2 : "SB"+"\s"+"\d+"+"\s"+"-"+"\s"+"AG"+"."+"."+"\s"+"\d"+"...\d+"
    list_2 = soup.find_all(string=re.compile("SB"+"\s"+"\d+"+"\s"+"-"+"\s"+"AG"+"."+"."+"\s"+"\d"+"...\d+"))
    
    #Case 3 : 
    liste_3 = soup.find_all(string=re.compile("SB"+"\s"+"\d+"+"\s"+"-"+"\s"+"AG"+".."+"\s"+"\d"+"............\d"))
    
    #Case 4 : 
    list_4 = soup.find_all(string=re.compile("SB"+"-..."))
    
    #Case 5 : 
    list_5 = soup.find_all(string=re.compile("SB"+"\s"+"\d+"+"\s"+"- AWG..."))
    
    # Clean list_2
    for i in range(len(list_2)) :
        list_2[i] =  re.sub("- AG\d+ \d ", '', list_2[i])
    
    for i in range(len(list_3)) :
        list_3[i] =  re.sub("- AGBM \d . AG....", '', list_3[i])
    
    for i in range(len(list_4)) :
        list_4[i] =  re.sub("-", ' ', list_4[i])
    
    for i in range(len(list_5)) :
        list_5[i] =  re.sub(" . AWGs", '', list_5[i])

    list = list_1+list_2+list_3+list_4+list_5

    return compute_list(list)

In [ ]:
# ------------------------------ IPCC ------------------------------ 

In [166]:
# Function that extract the list of all the IPCCs from a webpage and return a list containing all the COPs with their number, date and place

def extract_list_ipccs(soup):
    
    # find all the different IPCCs (not named the same way)
    
    #Case 1 :"IPCC-\d+ . "
    list = soup.find_all(string=re.compile("IPCC-\d+ . "))

    for i in range(len(list)) :
        list[i] =  re.sub("-", ' ', list[i])
        
    return compute_list(list)


In [ ]:
# ------------------------------ AGBM ------------------------------ 

In [199]:
# Function that extract the list of all the AGMBs from a webpage and return a list containing all the COPs with their number, date and place

def extract_list_agbms(soup):
    
    # find all the different AGBMs (not named the same way)    
    #Case 1 : "AGBM \d+ . \d+"
    list = soup.find_all(string=re.compile("AGBM \d+ . \d+"))
        
    return compute_list(list)

In [ ]:
# ------------------------------ UNFCCC WS ------------------------------ 

In [205]:
# Function that extract the list of all the AGMBs from a webpage and return a list containing all the COPs with their number, date and place
# A TERMINER - PAS DE NUMERO DE CONFERENCE MAIS DES LETTRES 
def extract_list_unfcccs(soup):
    
    # find all the different AGBMs (not named the same way)    
    #Case 1 : "AGBM \d+ . \d+"
    list = soup.find_all(string=re.compile("UNFCCC WS............."))
       
    return compute_list(list)

In [167]:
def combine_information_meetings(soup,meeting_type,extract_type):
    list_meetings = extract_type(soup)
    list_meetings_information = extract_details_meetings(soup,meeting_type)
    total = []
    for i in range(len(list_meetings)):
        (number,date,place) = list_meetings[i]
        list_meeting = list_meetings_information[i]
        
        for x in list_meeting:
            total.append((meeting_type,number,date,place,x[0],x[1],x[2],x[3]))
    return total